In [ ]:
# I used this for getting the model I wanted from the training site, and making into a TF model.
# I used lc0 and leela2onnx to convert it into a ONNX-model as an intermediary
# Note: if you want to do inference, the #1 way to do that is probably to just download the weight file of your choice
# and use the lc0-binary itself.
import urllib.request
urllib.request.urlretrieve("http://training.lczero.org/networks/?show_all=1", "networks.html")

In [2]:
from bs4 import BeautifulSoup

with open("networks.html") as f:
    soup = BeautifulSoup(f, 'html.parser')

network_iterations = [1, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2390]
results = []
for row in soup.find_all("tr"):
    for iter in network_iterations:
        iteration_string = str(30000 + iter)
        if iteration_string in row.contents[1]:
            results.append((iteration_string, row.contents[5].contents[0]['href']))

In [5]:
import urllib.request
import os
from tqdm import tqdm
# You could also download these manually if you want.
os.makedirs("networks", exist_ok=True)
opener = urllib.request.build_opener()
# Sneaky!
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)
iteration, href = results[0]
urllib.request.urlretrieve("http://training.lczero.org" + href, "networks/" + iteration + ".pb")

('networks/32390.pb', <http.client.HTTPMessage at 0x2cd03a94910>)

In [7]:
import tensorflow.keras as keras
import numpy as np

In [8]:
keras.backend.set_image_data_format('channels_first')

def block(last, i):
    x = keras.layers.Conv2D(256, (3, 3), padding="same", name="block{}/conv{}".format(i, 1), activation="relu")(last)
    x = keras.layers.Conv2D(256, (3, 3), padding="same", name="block{}/conv{}".format(i, 2), activation="linear")(x)
    add = keras.layers.Add()([x, last])
    rel = keras.layers.ReLU()(add)
    return rel, add


inputs = keras.Input((112, 8, 8))
last = keras.layers.Conv2D(256, (3, 3), padding="same", name="inputconv")(inputs)
last = keras.layers.ReLU()(last)
inter_outputs = [last]
for b in range(0, 20):
    last, inter = block(last, b)
    inter_outputs.append(inter)

policy_conv = keras.layers.Conv2D(32, (1, 1), activation="relu", padding="same", name="policy_conv")(last)
flatten = keras.layers.Flatten(data_format="channels_last")(policy_conv)
policy_output = keras.layers.Dense(1858, activation="linear", name="policy_dense")(flatten)

value_conv = keras.layers.Conv2D(32, (1, 1), activation="relu", padding="same", name="value_conv")(last)
flatten = keras.layers.Flatten(data_format="channels_last")(value_conv)
value_dense = keras.layers.Dense(128, activation="relu", name="value_dense_1")(flatten)
value_output = keras.layers.Dense(1, activation="tanh", name="value_output")(value_dense)

model = keras.Model(inputs, [policy_output, value_output])



In [10]:
import onnx

from onnx import numpy_helper
_model = onnx.load("network.onnx")
INTIALIZERS = _model.graph.initializer
weights = {}
for initializer in INTIALIZERS:
    W = numpy_helper.to_array(initializer)
    if len(W.shape) > 2:
        weights[initializer.name[1:]] = np.moveaxis(W, [0, 1], [-1, -2])
    else:
        weights[initializer.name[1:]] = W

In [12]:
for layer in model.layers:
    if layer.name == "value_dense_1":
        layer.set_weights([weights["value/dense1/matmul/w"], weights["value/dense1/add/w"]])
    elif layer.name == "value_output":
        layer.set_weights([weights["value/dense2/matmul/w"], weights["value/dense2/add/w"]])
    elif "value_conv" in layer.name:
        layer.set_weights([weights["value/conv/w/kernel"], weights["value/conv/w/bias"]])
    elif "policy_conv" in layer.name:
        layer.set_weights([weights["policy/conv/w/kernel"], weights["policy/conv/w/bias"]])
    elif "policy_dense" in layer.name:
        layer.set_weights([weights["policy/dense/matmul/w"], weights["output/policy/w"]])
    elif "conv" in layer.name:
        layer.set_weights([weights[layer.name + "/w/kernel"], weights[layer.name + "/w/bias"]])

In [13]:
model.save("models/tf_model")

INFO:tensorflow:Assets written to: models/tf_model\assets


INFO:tensorflow:Assets written to: models/tf_model\assets
